# Title: A _Sports Parent's_ Map to Olympic Victory


### Table of Contents:
- Introduction:
    - Project Summary and Goal
    - About the Data
- Data Importing and Cleaning
    - Web Scraping and formatting
    - Getting Coordinates for each country
- Data plotting and Visualizations
    - Creating the plots
    - Plotting on Folium map
- Conclusions 
    - Insights and possible improvements


### **Project Summary and Goal:**

We all have come across someone who is obsessed with succeeding at something: academics, their career, getting into the Guinness World book of Records. All is fine and good, but nothing comes close to the fanaticism of success at sports.

Sports success enjoys a wide audience: athletes want to be the next Michael Jordan, coaches want their teams to win, club managers and owners enjoy the profits of successful teams, betters place bets on success and of course the fans lose sleep over their teams' performance.
This project, however, will focus on parents who want nothing more than for their kids to be world-renonwned athletes. These parents are otherwise known as 'Crazy Sports Parents.' This project assumes that the location where an athelte is raised is a major indicator of their success as an athlete.

The goal of this project is to produce an interactive visual plot of Athletic medallists in the 2012 summer Olympics by country, gender and number of medallists. An assumption here is that Crazy Sports Parents are financially buoyant. Therefore this map serves as a one-stop visual aid to guide them on where to relocate to in order to increase their chances of raising successful athletics kids. 



### **About the data:** 

The data used for this project is the 2012 Summer Olympics medal winners from Wikipedia. (link: https://en.wikipedia.org/wiki/List_of_2012_Summer_Olympics_medal_winners). The website contains tables for each sport that lists every athelete who won a medal (Gold, Silver or Bronze), the country the athlete(s) is/are from and the specific sport they won the medal in. There is no numerical data in the table as it just lists name, country, medal type and the sport in question.


The data can be used to get the total number of winners in a sport by country and gender as the men's and women's tables, though identical, are separate. This can then be plotted on graphs to display which countries produce more winners in a specific sport. This will be helpful for parents seeking to relocate to increase their children's chances of also becoming Olympic medallists in a specific sport. This project focuses on just Athletics which consists of races, hurdles, jumps and throws all categorized under 'Athletics.' 

### **Data Importing and Cleaning:**

The data was obtained through scraping the Wikipedia website using REquests and BeautifulSoup modules. The Requests module was used to ping the website and get access to its content, while BeautifulSoup module was used to parse the the raw html data before it was rendered and loaded into clean pandas dataframes. 
The modules and code below were used to obtain the initial raw data from the website. The tables of interest are the men's and women's Athletics tables which are the 2nd and 3rd relevant tables in the website respectively. In Python however, indexing starts from 0, thus these become the 1st and 2nd tables respectively.

As there was no issue with continuously sending requests to the Wikipedia website to get this data, a delay method such as 'time.sleep(3)' to avoid a 404 error was not used.

In [16]:
# Web Scraping the data
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame


url = 'https://en.wikipedia.org/wiki/List_of_2012_Summer_Olympics_medal_winners'
page = requests.get(url)
soup = BeautifulSoup(page.content,'lxml')

# "wikitable plainrowheaders" class is specified to get the tables of interest.
# This is the table's class in the html code on the website.
table = soup.find_all('table', class_="wikitable plainrowheaders")
df_male = pd.read_html(str(table))[1]
df_female = pd.read_html(str(table))[2]

#display the first 5 lines of the men's table to show structure
df_male.head(5)


,Games,Gold,Silver,Bronze
0,100 metresdetails,Usain Bolt Jamaica,Yohan Blake Jamaica,Justin Gatlin United States
1,200 metresdetails,Usain Bolt Jamaica,Yohan Blake Jamaica,Warren Weir Jamaica
2,400 metresdetails,Kirani James Grenada,Luguelín Santos Dominican Republic,Lalonde Gordon Trinidad and Tobago
3,800 metresdetails,David Rudisha Kenya,Nijel Amos Botswana,Timothy Kitum Kenya
4,1500 metresdetails,Taoufik Makhloufi Algeria,Leonel Manzano United States,Abdalaati Iguider Morocco


As shown above, the data was parsed into pandas dataframes beautifully. However this is not helpful for plotting because we need the number of medallists by country, not the names. Below is the code written to scrape the website and return the number of athletes for each medal type in the table.

In [17]:
# Define a function to get the number of medalists by country. Apply it to both tables

url = 'https://en.wikipedia.org/wiki/List_of_2012_Summer_Olympics_medal_winners'
page = requests.get(url)
soup = BeautifulSoup(page.content,'lxml')
tables = soup.find_all('table', class_="wikitable plainrowheaders")

def numerical_data(raw_table):
    """ Returns the number of medallists from the wikipedia summer 2012 Olympics
    and parses it into a pandas dataframe."""
    medals = ['Gold', 'Silver', 'Bronze']
    data = []
    rows = raw_table.find_all('tr')[1::]
    for row in rows:
        cells1 = row.find_all('th')
        cells2 = row.find_all('td')
        cells =  cells1 + cells2
        for sub in range(0, len(cells[1::])):
            game = {} # Initialize dictionay to be converted to pd dataframe
            game['game'] = cells[0].text.replace('details', '').strip()
            subcell = cells[sub+1]
            game['medal'] = medals[sub]
            try:
                game['country'] = subcell.select_one('img[src*="Flag_"]').find_next_sibling('a').text
            except:
                game['country'] = ''
            try:
                people = subcell.find_all('a')
                game['count'] = len(people)-1
            except:
                people = []
                game['count'] = 0

            data.append(game)
    return pd.DataFrame(data)


# #For the Male table:
m_table = tables[1]
m_table = numerical_data(m_table)


# #For the female table:
f_table = tables[2]
f_table = numerical_data(f_table)


# Display a table to show format
f_table


,game,medal,country,count
0,100 metres,Gold,Jamaica,1
1,100 metres,Silver,United States,1
2,100 metres,Bronze,Jamaica,1
3,200 metres,Gold,United States,1
4,200 metres,Silver,Jamaica,1
...,...,...,...,...
64,Javelin throw,Silver,Germany,1
65,Javelin throw,Bronze,Germany,1
66,Heptathlon,Gold,Great Britain,1
67,Heptathlon,Silver,Germany,1



The table above looks much more promising in terms of plottable numerical values. However, the format is still not condusive for plotting. There are missing values in the country column. As the 'country' column is the index in this table, all rows with an empty 'country' value are removed. The table is also pivotted so that the medals are grouped by country. These are shown below:


In [18]:
import numpy as np

#Create a pivot table to create condusive format for the plotting/map visualization
def create_pivot(tab_to_piv):
    """ Returns a pivotted table 
    by grouping the medals by country"""
    if 'country' in tab_to_piv.columns:
        piv_table = pd.pivot_table(tab_to_piv, values='count', index =['country'], 
                                   columns = ['medal'], aggfunc=np.sum, fill_value=0)
    return piv_table


# Call the pivot function and drop rows with empty values in 'country column'
m_tab_piv = create_pivot(m_table).drop(index='')
f_tab_piv = create_pivot(f_table).drop(index='')

# Display a portion of the table to show format
m_tab_piv.head(5)

medal,Bronze,Gold,Silver
country,,,
Algeria,0,1,0
Australia,0,1,1
Bahamas,0,4,0
Botswana,0,0,1
China,1,1,1



The above table is almost ready for the analysis and visualiation. The 'country' column has to be a column of it's own, therefore the table should be re-indexed so that the country column becomes a separate column.


In [19]:
# Reset the index of the dataframe so that 'country' is a column and not the index
m_prelim_table = m_tab_piv.reset_index()
f_prelim_table = f_tab_piv.reset_index()

# Display a portion of the table to show format
m_prelim_table.head(5)

medal,country,Bronze,Gold,Silver
0,Algeria,0,1,0
1,Australia,0,1,1
2,Bahamas,0,4,0
3,Botswana,0,0,1
4,China,1,1,1




The table above is ready for plotting on a graph. 

### Getting the cooridnates for each country.
Now that a suitable table has been obtianed, the coordinates of the countries have to be obtained to enable geoplotting. 
The coordinates were obtained through scraping a table in the Google Developers website which gives the coordinates of every country: https://developers.google.com/public-data/docs/canonical/countries_csv



In [20]:
# Get the longitude and Latitude of the countries from scraping the google website. 
map_url = 'https://developers.google.com/public-data/docs/canonical/countries_csv'
map_page = requests.get(map_url)
soup = BeautifulSoup(map_page.content,'lxml')
m_table = soup.find_all('table')
map_table = pd.read_html(str(m_table))[0]


# Display to show format
map_table.head(5)

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla



The column headers of this newly scraped table 'map_table' were renamed to enable merging with our 'prelim' tables further down the line.


In [21]:
# Rename the fields to enable merging tables with the olympic table

if 'name' in map_table.columns and 'country' in map_table.columns:
    map_table.rename({'name': 'country','country':'code'}, axis=1, inplace=True)


# Display to show format
map_table.head(5)

,code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [22]:
#Use pandas 'merge' to left join the two tables on the country column

def merge_table(final_table):
    """Takes in a table and left merges it with the map_table
    dataframe on the 'country' column."""
    big_table = final_table.merge(map_table, on = 'country', how = 'left')
    big_table.drop(['code'], axis=1, inplace = True)
    return big_table


m_final_table = merge_table(m_prelim_table)
f_final_table = merge_table(f_prelim_table)

m_final_table.head(15)

,country,Bronze,Gold,Silver,latitude,longitude
0,Algeria,0,1,0,28.033886,1.659626
1,Australia,0,1,1,-25.274398,133.775136
2,Bahamas,0,4,0,25.034280,-77.396280
3,Botswana,0,0,1,-22.328474,24.684866
4,China,1,1,1,35.861660,104.195397
5,Cuba,1,0,0,21.521757,-77.781167
6,Czech Republic,1,0,0,49.817492,15.472962
7,Dominican Republic,0,1,1,18.735693,-70.162651
8,Estonia,1,0,0,58.595272,25.013607
9,Ethiopia,1,0,1,9.145000,40.489673


Now that both tables have been merged and the irrelevant 'code' column has been removed. We can see that there is an issue with the data. The Great Britain (GB) 'longitude' and 'latitude' cells are empty. This is because the coordinates for GB in map_table are stored under the country name 'United Kingdom' (UK).

The table will also be scanned for any other NaN values in the data.

In [23]:
# Check if the tables have any null values and count the number of NaNs.

f_final_table.isnull().values.any()
f_NaN = f_final_table.isnull().sum().sum()

m_final_table.isnull().values.any()
m_NaN = m_final_table.isnull().sum().sum()

print(f_NaN, m_NaN)

2 2



There are 2 null values in both tables which correspond to the NaN values in the GB long and lat columns. The assumption made is that UK is equivalent to GB here, thus the coordinates for GB will be filled with the UK coordinates in the map_table. This is done below:


In [24]:
#Get the UK coordinates from map_table and assign them to variables

uk_coords = map_table.loc[map_table['country'] == 'United Kingdom']
print(uk_coords)

print('\n')

# Use output from above to index the dataframe and get the right coordinates (i.e. row 73)
uk_lat = map_table.at[73,'latitude']
uk_long = map_table.at[73,'longitude']
print('uk_lat: ', uk_lat, '\n' 'uk_long', uk_long)

   code   latitude  longitude         country
73   GB  55.378051  -3.435973  United Kingdom


uk_lat:  55.378051 
uk_long -3.435973


In [25]:
#Replace the 'Great Britain' NaNs with the UK coordinates

def coordinate_sub(df):
    """Replaces the GB NaN values in long and lat
    columns with the UK long and lat values from map_table."""
    df['latitude'] = df['latitude'].fillna(uk_lat)
    df['longitude'] = df['longitude'].fillna(uk_long)
    return df

coordinate_sub(f_final_table).head(5)
coordinate_sub(m_final_table).head(5)

#Display the GB row for one dataframe.
m_final_table.iloc[13]

country      Great Britain
Bronze                   0
Gold                     3
Silver                   0
latitude           55.3781
longitude         -3.43597
Name: 13, dtype: object

Now that the above dataframes have been cleaned up, a few more manipulations will be done to plot an even more informative graph. 

The total number of winners for each country in both male and female tables will be added to the table. This is done by creating a new column and populating it with the sum of all medallists for that country. 

With this new data, we can draw parents' attention to the conutry with the most winners. The goal wasn't to lead them to the most Gold, it was to lead them to the most medals, hence the addition. If a child qualifies for the Olympics, then they're world-class anyway. Winning a medal is the icing on the cake, so the assumption is that the parents would be happy with even a Bronze (hopefully).

The tables will then be joined by country in order to have both the women's and men's data in one cohesive table for plotting.

In [26]:
# Create a large table with 'total' columns.

# Create a 'total' columns in both dataframes.
f_final_table['female_total'] = f_final_table['Bronze'] + f_final_table['Gold'] + f_final_table['Silver']
m_final_table['male_total'] = m_final_table['Bronze'] + m_final_table['Gold'] + m_final_table['Silver']

# Join womens table on the mens table, fill 'na' with 0 and align female_total data type to male_total data type
olympic_table = pd.merge(m_final_table[['country','latitude','longitude','male_total']],
                         f_final_table[['country','female_total']],on='country', how='left').fillna(0)

olympic_table['female_total'] = olympic_table['female_total'].astype(int)

olympic_table['all_total'] = olympic_table['female_total'] + olympic_table['male_total']


#Insert colour column to use for the points on the graph
olympic_table['colour'] = np.where(olympic_table.all_total < 10, '#0077BB', 
                                   np.where(olympic_table.all_total <= 20, '#FFC20A', '#EE3377'))

#Display the table
olympic_table.head(5)

,country,latitude,longitude,male_total,female_total,all_total,colour
0,Algeria,28.033886,1.659626,1,0,1,#0077BB
1,Australia,-25.274398,133.775136,2,1,3,#0077BB
2,Bahamas,25.034280,-77.396280,4,0,4,#0077BB
3,Botswana,-22.328474,24.684866,1,0,1,#0077BB
4,China,35.861660,104.195397,3,6,9,#0077BB


As shown above, the final overall table now has both male and female data and an overall 'total' column as well. A colour column has been inserted as this will be used for the visualization. 

The colours are used to distinguish countries by the total number of medallists from that country. The colours chosen are suitable for all audiences to consider people with visual impairments or colour blindness, and were gotten from this website: https://personal.sron.nl/~pault/#sec:qualitative

The rubric for colour coding (as shown in the code cell above) is:

- Magenta: Countries with > 20 medalists
- Yellow: Countries with < 20 but > 10 medalists 
- Blue: Countries with < 10 medalists

The colors were intentionally chosen to have as much contrast as possible to enable readability. Also, if parents will be relocating from their home country to increase chances of their kids' Olympic success, we might as well help save them some money on an Optometrist visit.


## Data Plotting and Visualizations:

### Creating the plots
The module used to create and apply the graphs for this visualization is the 'vincent/vega' module (which 'coincidentally' is the name of the main character from 'Pulp Fiction.' I'm sure John Travolta would approve). 

The graphs are created for each country (each row in the final dataframe) with each bar representing the number of male and female medallists from that country. This will also help parents decide which country to relocate to depending on their child/children's gender.

One of the graphs was displayed after the code below.

In [27]:
import vincent

vincent.core.initialize_notebook()

#Create Vincent bar charts for all rows to visualize the medalists by gender
bar_list = []
for row in range(0,len(olympic_table)):
    bar = vincent.GroupedBar(olympic_table.iloc[row][['male_total','female_total']],width=600, height=300)
    bar.axis_titles(x='Gender', y='Number of medalists')
    bar.colors(brew='Spectral')
    bar.width=200
    bar.height = 300
    bar.legend(title=str(olympic_table.iloc[row]['country'])) #Use this place country name in each graph
    bar_list.append(bar)

# Display a graph
bar_list[1]

### Plotting on Folium map

Now we will plot all data points (graphs) on a geographical plot! Using the 'folium' module, a map was created. This map tile (Stamen Terrain) was chosen because is is comparitvely clearer, not too busy and has the country names clearly printed(which is another advantage for the parents because we would like this to be as readable as possible for them).

The folium map plotting is below. Each country is plotted with a circle marker that corresponds to the total number of Athletic Olympic medallists from that country. 

The 'branca' module was used to create and include a colour map as a legend to the visualization. This makes referencing quicker.

There are 2 plots. The first is without clustering and it gives immediate information on what region to focus on based on the color density of the region. 

The second is the same as the first, however it clusters the countries which have medallists and are in close proximity to each other. It is also an overall neater graph. The user(parent) would appreciate it's neatness, but would have to zoom in to get a clearer picture of what country to start learning the language of. 

In [28]:
import folium
import folium.plugins as plugins
import branca

#Pick coordinates of any country to initialize plot
US_coords = (37.090240, -95.712891)
olympic_map = folium.Map(location=US_coords, max_bounds = True, tiles='Stamen Terrain', zoom_start=2)

    
for a, row in olympic_table.iterrows():
    folium.CircleMarker(location=[row[1],row[2]], color = row[6], radius=15, fill_opacity =0.65,
    popup=folium.Popup(max_width=300).add_child(folium.Vega(bar_list[a], height=350, width=650)), 
    fill=True).add_to(olympic_map)


# Add Colour Legend to the map
colormap = branca.colormap.LinearColormap(colors=['#0077BB','#FFC20A','#EE3377'], index=[-600,8,1200],vmin=-600,vmax=1200).scale(-600,1200)
colormap = colormap.to_step(index=[0, 10, 20, 45])
colormap.caption = 'Total number of Athletics medalists in 2012 Olympics'
colormap.add_to(olympic_map)
olympic_map

### The Final Plot!

Below is the final plot with a bit of an improvement to the plot above. This map has clustering and is thus neater. 

The plot is dynamic and interactive with a cursor hand that allows the user to move around. Clicking on a circle marker reveals the plot for the selected country that shows the comparison of male vs female medalists. 

Overall, the map is a tool that is color coded by number of medallists, displays a graph showing the comparison of male vs female medallists and is plotted directly on the map wich allows for a quick visualization.

In [29]:
#Pick Random coordinates from list to initialize plot
US_coords = (37.090240, -95.712891)
olympic_map2 = folium.Map(location=US_coords, max_bounds = True, tiles='Stamen Terrain', zoom_start=2)

#initialize the marker cluster plugin for folium
mc2=plugins.MarkerCluster()
    
for a, row in olympic_table.iterrows():
    folium.CircleMarker(location=[row[1],row[2]], color = row[6], radius=15, fill_opacity =0.75,
    popup=folium.Popup(max_width=300).add_child(folium.Vega(bar_list[a], height=350, width=650)), 
    fill=True).add_to(mc2)
    
# Display the map
mc2.add_to(olympic_map2)
colormap.add_to(olympic_map2)
olympic_map2

# Conclusion

### Insights:

Parents can use this graph to see what countries have the most number of olypmic medallists for Athletics.

A surprising find is that the country with the most number of Athletic medallists is the United States and there are more female medallists there than male medallists, revealing that women are dominating the Olympic Athletics space in the US. The same can be said for China with twice as many women winning Athletics medals than men. This is a very great find which is contrary to the overall pop culture image of who dominates the Athletics scene. 

This insight also insinuates the possibility that prior to the 2012 Olympics, The US and China put more resources towards their female athletes than other countries did, hence the much greater number of female medallists from these countries. 

This is one year worth of data however, therefore this is not fully representative. Another finding is that men overall, do better than women in Athletics sports worldwide. European countries are leading in terms of number of countries who win Athletics medals overall and regions such as West Africa have no Athletics medallists.

If parents want to use 2012 Olmypic data to decide where relocate to improve their child's chances of becoming an Olympic medallist in Athletics, they should consider the United States, Jamaica, Trinidad and Tobago or Kenya as their top picks. I personally would pick Jamaica or Trinidad because it is Summer all year round and while my child practices, I can be at the beach.

### Final Note:
If you are a Crazy Sports parent who happens to stumble across this data analysis, please visit this link:
https://yourteenmag.com/sports/wacko-sports-parent

My advice is: 'Don't be a Crazy Sports Parent! Let your kids live, enjoy their childhood and be guided by intuition to their dreams!'


### Possible Improvements:

As with any piece of work, there are always improvements that can be made. I will list the top 4 for the tools/modules used in this project and the analysis itself.
- Vincent currently does not seem to have functionality to give a graph a title. Therefore the 'legend' functionality was used. The graphs could be made neater if there was a 'title' functionality
- Folium graphs are powerful visualization tools. However, the maps loop to infinity when zoomed out, irrespective of if bounds are introduced (as is done above). The max bounds only works within the specified zoom.
- Branca colour legends are also great tools, however they are positioned in the top right by default. The best position here would have been bottom left as that is blank space. Nevertheless, the legend position is ok in these plots because there is no pertinent informtion in the top right of this plot. 
- More data could be used to get a more informative analysis. As this project is to showcase the use of plots for visualization, the size of data used is ok. However, more data would give a better picture into countries that produce winning athletes based on their historical number of medallists therefore giving more accurate and statistically complete information.